In [1]:
import json
import os
import requests

In [2]:
def printResponse(r):
    print('{} {}\n'.format(json.dumps(r.json(), sort_keys=True, indent=4, separators=(',', ': ')), r))

In [3]:
token_file = os.path.join(os.path.expanduser("~/"), ".jira_auth")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

In [4]:
SERVER = "https://confluence.lsstcorp.org/"
SIM_SPACE="SIMS/LSST+Simulations+Framework"
JSON_TYPE={'Content-Type':'application/json'}

In [5]:
r = requests.get(SERVER + '/rest/api/content',
    params={'title' : 'LSST Simulation Framework'},
    auth=(uname, pwd))
#printResponse(r)
parentPage = r.json()['results'][0]

In [6]:
parentPage["id"]

'1507517'

In [7]:
pageData = {"type": "page", "title": "New page", "ancestors": [{"id": parentPage["id"]}], 
            "space": {"key": "SIM"}, 
            "body": {"storage": {"representation": "storage", "value":"<p>This is a new page</p>"}}}

In [8]:
r = requests.post(SERVER + '/rest/api/content',
                  data=json.dumps(pageData),
                  auth=(uname, pwd),
                  headers=(JSON_TYPE))
#printResponse(r)

In [9]:
pageReturn = r.json()
print(pageReturn["id"])
print(pageReturn["title"])
print(pageReturn["version"]["number"])

56689995
New page
1


In [12]:
updatePage = {"id": pageReturn["id"], "type": "page", "title": pageReturn["title"],
              "space": {"key": "SIM"},
              "body": {"storage": {"value": "<p>This is the updated text for the new page.</p>", 
                                   "representation": "storage"}},
              "version": {"number": pageReturn["version"]["number"] + 1}}

In [13]:
r = requests.put(SERVER + '/rest/api/content/' + str(pageReturn["id"]),
                 data=json.dumps(updatePage),
                 auth=(uname, pwd),
                 headers=(JSON_TYPE))
#printResponse(r)

In [14]:
r.status_code

200

In [15]:
r = requests.get(SERVER + '/rest/api/content/' + str(pageReturn["id"]) + "/history",
    params={"title" : pageReturn["title"]},
    auth=(uname, pwd))
#printResponse(r)

In [16]:
pageHistory = r.json()
print(pageHistory["lastUpdated"]["number"])

2


In [17]:
updatePage2 = {"id": pageReturn["id"], "type": "page", "title": pageReturn["title"],
               "space": {"key": "SIM"},
               "body": {"storage": {"value": "<p>Updated the text for the new page again.</p>", 
                                    "representation": "storage"}},
               "version": {"number": pageHistory["lastUpdated"]["number"] + 1}}

In [18]:
r = requests.put(SERVER + '/rest/api/content/' + str(pageReturn["id"]),
                 data=json.dumps(updatePage2),
                 auth=(uname, pwd),
                 headers=(JSON_TYPE))
#printResponse(r)

In [19]:
r = requests.delete(SERVER + '/rest/api/content/' + str(pageReturn["id"]),
                    auth=(uname, pwd))
print(r.status_code)

403
